In [2]:
# Dependencies
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import requests
import json
import gmaps
from config import g_key
gmaps.configure(api_key=g_key)

In [3]:
shark_df = pd.read_csv("shark_geo.csv")

In [ ]:
# Create a map with markers
marker_locations = shark_df[['lat', 'lng']]

# Create a marker_layer 
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig